# Join all data from previous steps to one file

In [2]:
import pandas as pd
import numpy as np

In [4]:
transcriptsv1 = pd.read_csv("transcripts_dataset_v1.csv.gz", compression="gzip")
transcriptsv2 = pd.read_csv("transcripts_dataset_v2.csv.gz", compression="gzip")
metadata = pd.read_csv("metadata.csv.gz", compression='gzip')

In [5]:
print('Transcipt v1 shape:', transcriptsv1.shape)
print('Transcipt v2 shape:', transcriptsv2.shape)
print('Metadata shape:', metadata.shape, '\n')

# print(list(transcriptsv1.columns))
# print(list(transcriptsv2.columns))
# print(list(metadata.columns))

# print('\n')

# print('Transcript v1 null check:\n', transcriptsv1.isna().sum(), '\n')
# print('Transcript v2 null check:\n', transcriptsv2.isna().sum(), '\n')
print('Metadata null check:\n', metadata.isna().sum()) # show_name 2 nulls, episode_description 205 nulls, category 6000, pubdate 20030


Transcipt v1 shape: (39892, 6)
Transcipt v2 shape: (65468, 6)
Metadata shape: (105360, 14) 

Metadata null check:
 show_uri                       0
show_name                      0
show_description               2
publisher                      0
language                       0
rss_link                       0
episode_uri                    0
episode_name                   0
episode_description          205
duration                       0
show_filename_prefix           0
episode_filename_prefix        0
category                    6000
pubdate                    20030
dtype: int64


In [6]:
# concatenate transcripts dataframes
transcripts = pd.concat([transcriptsv1, transcriptsv2])
print(transcripts.shape)

# subset metadata
metadata = metadata[["show_name", "show_description", "publisher", "language", "episode_name", "episode_description", "duration", "show_filename_prefix", "episode_filename_prefix", 'category', 'pubdate']]
metadata = metadata.rename({"episode_filename_prefix": "episode_id", "show_filename_prefix": "show_id"}, axis="columns")  # rename cols

# remove episode_id suffix 
transcripts.episode_id = transcripts.episode_id.apply(lambda x: x.replace(".json", ""))
transcripts = transcripts.drop("Unnamed: 0", axis=1)

# remove whitespace
metadata.episode_id = metadata.episode_id.apply(lambda x: x.strip())

(105360, 6)


In [7]:
# join data
full_dataset = transcripts.join(metadata.set_index("episode_id"), on="episode_id", rsuffix="_trans")
full_dataset.isna().sum()

show_id                    0
episode_id                 0
transcript                 0
avg_confidence             0
word_count                 0
show_name                  0
show_description           2
publisher                  0
language                   0
episode_name               0
episode_description      205
duration                   0
show_id_trans              0
category                6000
pubdate                20030
dtype: int64

In [8]:
print(full_dataset.shape)
# full_dataset.head(5)

(105360, 15)


In [ ]:
# Null inspection
full_dataset[full_dataset.isnull().any(axis=1)] # metadata discrepancies, episode_id present for all instances

### Correct mistake in word count from directory walk

In [10]:
# word_count was actually char count 
full_dataset = full_dataset.rename(columns={"word_count": "char_count"})

# count words
full_dataset["word_count"] = full_dataset.transcript.apply(lambda x: len(x.split(" ")))

In [11]:
languages_not_keep = ["['nl-BE']","['hi']","['id']","['ml']","['ms']","['pt']","['en-JM']","['en-IN']","['ga']","['ta']","['es']"]
languages = list(set(full_dataset.language.unique()) - set(languages_not_keep))

In [12]:
full_dataset = full_dataset[full_dataset['language'].isin(languages)]
print(full_dataset.shape)
full_dataset.language.value_counts()

(105228, 16)


['en']       79452
['en-US']    20264
['en-GB']     2335
['en-AU']     1377
['en-CA']      962
['en-PH']      368
['en-IE']      243
['en-NZ']      141
['en-ZA']       86
Name: language, dtype: int64

In [13]:
# Save the DataFrame as a gzip-compressed CSV file
# full_dataset.to_csv('transcripts_dataset_final.csv.gz', compression='gzip', index=False)

## Filter out Education and Sports

In [3]:
transcripts_full = pd.read_csv("transcripts_dataset_final.csv.gz", compression="gzip")
print(transcripts_full.columns)
transcripts_full.shape

Index(['show_id', 'episode_id', 'transcript', 'avg_confidence', 'char_count',
       'show_name', 'show_description', 'publisher', 'language',
       'episode_name', 'episode_description', 'duration', 'show_id_trans',
       'category', 'pubdate', 'word_count'],
      dtype='object')


(105228, 16)

In [5]:
education_transcripts = transcripts_full[transcripts_full.category == 'Education']
education_transcripts.shape
# education_transcripts.to_csv('education_transcripts.csv.gz', compression='gzip')

In [4]:
sports_transcripts = transcripts_full[transcripts_full.category == 'Sports']
sports_transcripts.shape
# sports_transcripts.to_csv('sports_transcripts.csv.gz', compression='gzip')